### t-Test Demonstration

##### Vorlage:
https://colab.research.google.com/github/pachterlab/bibecs183/blob/master/Colab_Notebooks/p_value_walk.ipynb#scrollTo=ZH2kLPq7Ol5c
*(Programmiersprache: R)*

#### Reinterpretation in python 3:
Interaktive Visualisierung statistischer t-Tests.

**interaktive Schaltflächen:**
* Testauswahl: Ein- oder Zweistichproben-t-Test oder abhängiger t-Test
* Normalverteilung: Stichprobengröße, Erwartungswert, Standardabweichung
* Grenzwert: Schieberegler zur visualisierung des Grenzwertes
* Neubrechnung: neue Werte mit den aktuellen Einstellungen berechnen
* Diagramm: test-Ergebnisse über der Stichprobengröße

**Berechnungsablauf:**
- basierend auf der Vorlage werden iterativ 2 Vektoren mit beliebigen, normalveteilten Elementen aus einer unbekannten Grundgesamtheit gefüllt.
- in jeder Iteration werden die Elemente die bis dahin in den Vektoren vorhanden sind, in einem t-Test geprüft.
- Die Vektorgröße wird über den Schieberegler 'Stichprobengröße' bestimmt.


In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets as widgets

%matplotlib nbagg

In [ ]:
# während der Berechnung der Stichproben und der Durchführung des t-Tests werden Warnungen ausgegeben.
# Diese werden hier ausgeschalten
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# interaktive Schaltflächen initialisieren
#
# Stichprobengröße
style = {'description_width': 'initial'}
n_total = widgets.IntSlider(min=50, max=5000, value=1000, description="Stichprobengröße", style=style)
# Grenzwert des T-Tests
limit = widgets.FloatSlider(min=0.0, max= 2.0, value=0.05, step=.05, orientation="vertical", description="Grenzwert")
#
testForm = ['Einstichproben-t-Test','Zweistichproben-t-Test','Abhängiger-t-Test']
testDropdown = widgets.Dropdown(description='Testform auswählen', options=testForm, value='Zweistichproben-t-Test')
#
# ERwartungswert und Standrdabweichung der Standardverteilung
mu = widgets.FloatSlider(min=-15, max=15, value=0.0, description="µ:", tooltip="Erwartungswert der Normalverteilung")
sigma = widgets.FloatSlider(min=0.1, max=5, value=1, description='$\sigma$:', tooltip="Standardabweichung")
#
# buttons
# 
BtnReload = widgets.Button(description="neu berechnen",button_style='info', tooltip="berechne neue Werte mit den aktuellen Einstellungen")
BtnReset = widgets.Button(description="Reset", button_style='warning', tooltip="auf Standardeinstellungen zurücksetzen")
#
# output info
info_1 = widgets.Label(value="")
info_2 = widgets.Label(value="")# keine Mehrzeilige Ausgabe möglich, daher 2 Schaltflächen

In [ ]:
# Einstichprobentest https://de.wikipedia.org/wiki/Einstichproben-t-Test
#
# Einstichproben-t-Test für normal-verteilte Stichprobe durchführen
# - basierend auf der Vorlage werden iterativ 2 Vektoren mit beliebigen, normalveteilten Elementen aus einer unbekannte Grundgesamtheit gefüllt.
# - Die Vekorgröße (Stichprobengröße) wird über 'n_total' bestimmt.
# - in jeder Iteration werden die Elemente die bis dahin in den Vektoren vorhanden sind, in einem t-Test geprüft.
# 
# Berechnungsmethoden:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html#scipy.stats.ttest_1samp
# - beim Einstichproben-t-Test wird zweiseitig auf den Erwartungswert, der auch für die Bestimmung der Elemente gilt, geprüft.
#
def one_sample_t_test(samples, mu, sigma):
    #
# Der t-Test liefert ab einer Stichprobengröße > 3 verwendbare Ergebnisse.
# Die Vektoren werden um diese Elemente erweitert
    #   
    x=np.zeros(samples+3) #Stichprobenvektor
    p= np.array([np.zeros(samples+3),np.zeros(samples+3)], ndmin=2)
    p[0]=np.zeros(samples+3) #testergebnis
    p[1]=np.zeros(samples+3)
    for i in range(2, (samples+3) ,1):
        x[i] = stats.norm.rvs(mu, sigma)        
        p[0,i] = stats.ttest_1samp(x[1:i], mu).pvalue        

#     In python ist array-objekt unveränderbar; ein einzelnes Element kann nicht gelöscht werden. Man muss ein neues array-objekt definieren.
    pNew = np.delete(p[0],[0,1,2])
    return pNew

In [ ]:
# Zweistichproben-t-Test https://de.wikipedia.org/wiki/Zweistichproben-t-Test
#
# - basierend auf der Vorlage werden iterativ 2 Vektoren mit beliebigen, normalveteilten Elementen aus einer unbekannte Grundgesamtheit gefüllt.
# - Die Vekorgröße (Stichprobengröße) wird über 'n_total' bestimmt.
# - in jeder Iteration werden die Elemente die bis dahin in den Vektoren vorhanden sind, in einem t-Test geprüft.
#
# Berechnungsmethoden:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
# - der Zweistichproben-t-Test prüft, ob der Mitttelwert der beiden Stichproben gleich ist. 
#   Die absoulte Abweichung wird als Ergebnis gespeichert.
#
def independent_t_test(samples,mu, sigma):
    #
# Der t-Test liefert ab einer Stichprobengröße > 3 verwendbare Ergebnisse.
# Die Vektoren werden um diese Elemente erweitert
    #
    x=np.zeros(samples+3) #Stichprobenvektor_1
    y=np.zeros(samples+3) #Stichprobenvektor_2
    p=np.zeros(samples+3) #testergebnis
    for i in range(2, (samples+3) ,1):
        x[i] = stats.norm.rvs(mu, sigma,size=1)
        y[i] = stats.norm.rvs(mu, sigma,size=1)
        p[i] = stats.ttest_ind(x[1:i], y[1:i], equal_var = True).pvalue
        #
#     In python ist array-objekt unveränderbar; ein einzelnes Element kann nicht gelöscht werden. Man muss ein neues array-objekt definieren.
    pNew = np.delete(p,[0,1,2])
    return pNew

In [ ]:
# Abhängiger-t-Test 
#
# - basierend auf der Vorlage werden iterativ 2 Vektoren mit beliebigen, normalveteilten Elementen aus einer unbekannte Grundgesamtheit gefüllt.
# - Die Vekorgröße (Stichprobengröße) wird über 'n_total' bestimmt.
# - in jeder Iteration werden die Elemente die bis dahin in den Vektoren vorhanden sind, in einem t-Test geprüft.
#
# Berechnungsmethoden:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
# - der abhängige-t-Test prüft, ob der Mitttelwert von abhängigen oder wiederholten Stichproben gleich ist.
#   Die absoulte Abweichung wird als Ergebnis gespeichert.

def paired_t_test(samples, mu, sigma):
    #
# Der t-Test liefert ab einer Stichprobengröße > 3 verwendbare Ergebnisse.
# Die Vektoren werden um diese Elemente erweitert
    #
    x=np.zeros(samples+3)   #store x-values
    y=np.zeros(samples+3) #store y-values
    p=np.zeros(samples+3) #store p-values
    for i in range(2, (samples+3) ,1):
        x[i] = stats.norm.rvs(mu, sigma)
        y[i] = stats.norm.rvs(mu, sigma)
        p[i] =  stats.ttest_rel(x[1:i], y[1:i]).pvalue
#     In python ist array-objekt unveränderbar; ein einzelnes Element kann nicht gelöscht werden. Man muss ein neues array-objekt definieren.
    pNew = np.delete(p,[0,1,2])
    return pNew

In [ ]:
#
# Visualisierung
#
# - Diagramm zeichnen:
fig, ax = plt.subplots()
#
limit_old = limit.value
pVal = independent_t_test(n_total.value, mu.value, sigma.value)
#
# 1. callback funktion definieren die ausgeführt wird, wenn sich der Wert eines Schalters ändert
def update_view(*args):
    global limit_old
    global pVal
    
    # Testmethode
    testMode = testDropdown.value
    #
    # Neue Werte berechnen
    # - wenn sich nur der Grenzwert ändert, soll keine Berechnung gemacht werden, 
    #   nur die Grenzwertlinie neu gezeichnet werden.
    if limit_old == limit.value:
        # Einstichproben-t-Test
        if testMode == testForm[0]:
            pVal_new = one_sample_t_test(n_total.value, mu.value, sigma.value)
        # Zweistichproben-t-Test
        if testMode == testForm[1]:
            pVal_new = independent_t_test(n_total.value, mu.value, sigma.value)
        # Abhängiger-t-Test
        if testMode == testForm[2]:
            pVal_new = paired_t_test(n_total.value, mu.value, sigma.value)
        pVal = pVal_new
    
    
    pMin = pVal.argmin()
    pUnderLimit = ((pVal < limit.value).nonzero()[0] + 10)
    #
    #----------------------------------------------
    ax.clear()   
    ax.plot(pVal, 'r-', lw=3)
    ax.set_xlim(0, n_total.value)
    ax.set_ylim(0,1.5)
    ax.set_ylabel("p-value")
    ax.set_xlabel("sample size")
    #
    # horizontale Linie auf Höhe des Grenzwertes zeichnen
    ax.axhline(y=limit.value, xmin=0, xmax=n_total.value, ls='--', color="darkgrey", linewidth=2)
    #
    # vertikal Linien zeichen für den minimalen Wert und an der Stelle an der das erste Mal der Grenzwert unterschritten wurde
    yLim = plt.ylim()
    yPos = yLim[0] + yLim[-1] / 2
    bbox_props = dict(boxstyle="round", fc=(0.1, 0.1, 1), ec="0.5", alpha=0.9)
    if pMin:
        ax.axvline(x=pMin, ymin=0, ymax=0.5, ls='-', color="blue",lw=2)
        # beschriftung:        
        ax.annotate('minimum', xy=(pMin,yPos), xycoords='data',\
                    xytext=(-90,-50), textcoords='offset points',\
#                     ha = "center", va= "center",\
                    bbox=dict(boxstyle="round", fc=(0.1,0.1,1), ec="0.5", alpha=0.9),\
                    arrowprops=dict(arrowstyle="->", connectionstyle="arc,angleA=0, armA=50,rad=10"))
    #
    if (len(pUnderLimit) >  0):
        ax.axvline(x=(pUnderLimit[0] + 10), ymin=0, ymax=0.5, ls='-', lw=2, color="darkgreen")
        ax.annotate('first time < limit',\
                    xy=(pUnderLimit[0] + 10,yPos),\
                    xycoords='data', xytext=(-90,50),\
#                     ha = "center", va= "center",\
                    textcoords='offset points',\
                    bbox=dict(boxstyle="round", fc=(0.2,0.8,0.3), ec="0.5", alpha=0.9),\
                    arrowprops=dict(arrowstyle="->", connectionstyle="arc,angleA=0, armA=50,rad=10"))        
    #
    # x-axis ticks
    ax.set_xticks(np.arange(10, (n_total.value+10), (n_total.value/4)))
    #   
    plt.show()
    # --------------------
    # Ausgabetext
    if pMin:
        s1 = "Das kleinste Testergebnis wurde an der Stelle {} gefunden.".format(pMin)
        info_1.value = ""
        info_1.value = s1
    #
    if (len(pUnderLimit)> 0):
        s2 = "Das erste mal wurde das Limit von {} an der Stelle {} unterschritten.".format((limit.value),(pUnderLimit[0] + 10))
    else:
        s2 = "Kein Testergebnis hat das Limit von {} unterschritten".format(limit.value)
    #
    info_2.value = ""
    info_2.value = s2
    
    limit_old = limit.value
    
def reset_controls(*btn):
    # Reset Button:
    n_total.value = 1000
    limit.value = 0.05
    mu.value = 0
    sigma.value = 1
    info_1.value = ""
    info_2.value = ""
    testDropdown.value = 'Zweistichproben-t-Test'
    update_view()

#--------------------------------------------------------
# 2. callback funktion den einzelnen Schaltflächen mit der 'observe'-methode zuordnen
n_total.observe(update_view, 'value')
mu.observe(update_view,'value')
sigma.observe(update_view,'value')
limit.observe(update_view, 'value')
testDropdown.observe(update_view, 'value')
BtnReload.on_click(update_view)
BtnReset.on_click(reset_controls)

#--------------------------------------------------------
# Anwendung starten
#
# Diagram einmal zeichnen
update_view()
#
# Schaltflächen mit 'widgets.VBox / .HBox' ordnen
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch')
#
widgets.VBox([info_1,info_2,\
              widgets.HBox([limit, testDropdown, widgets.VBox([BtnReload,n_total,mu,sigma])]),\
              BtnReset],\
             layout = box_layout)